In [1]:
import torch

In [6]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
z = x + y
end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

print(start.elapsed_time(end))

1.2739520072937012


In [10]:
x = torch.randn((1, 1), requires_grad=True).to('cuda')
with torch.autograd.profiler.profile(enabled=True) as prof:
    for _ in range(100):  # any normal python code, really!
        y = x ** 2
        y += x
print(prof.key_averages().table(sort_by="self_cpu_time_total"))

---------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------  ------------  ------------  ------------  ------------  ------------  ------------  
            aten::pow        46.11%       4.897ms        60.97%       6.475ms      64.750us           100  
     cudaLaunchKernel        27.99%       2.973ms        27.99%       2.973ms      14.865us           200  
           aten::add_        25.84%       2.744ms        39.03%       4.145ms      41.450us           100  
    aten::result_type         0.04%       4.000us         0.04%       4.000us       0.040us           100  
             aten::to         0.02%       2.000us         0.02%       2.000us       0.020us           100  
---------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 10.620m

STAGE:2023-04-17 10:55:58 525:525 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-04-17 10:55:58 525:525 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-04-17 10:55:58 525:525 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [11]:
x = torch.randn((1, 1), requires_grad=True).to('cuda')
with torch.profiler.profile(activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
    for _ in range(100):  # any normal python code, really!
        y = x ** 2
        y += x
# print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=10）)

print(prof.key_averages(group_by_input_shape=True).table(sort_by="self_cpu_time_total"))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       cudaLaunchKernel        47.06%       7.487ms        47.06%       7.487ms      37.435us       0.000us         0.00%       0.000us       0.000us           200  
                                              aten::pow        33.12%       5.269ms        68.98%      10.976ms     109.760us     214.000us        51.69%     214.000us       2.140us           100  
         

STAGE:2023-04-17 10:58:11 525:525 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-04-17 10:58:11 525:525 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-04-17 10:58:11 525:525 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
